In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("default")
    .getOrCreate()
)

#https://www.bcb.gov.br/api/servico/sitebcb/historicotaxajurosdiario/TodosCampos?filtro=(codigoSegmento%20eq%20%272%27)%20and%20(codigoModalidade%20eq%20%27211101%27)%20and%20(InicioPeriodo%20eq%20%272025-03-24%27)

In [ ]:
PRODUTOS = [
    {
        "nome": "Aquisição de outros bens",
        "codigoSegmento": "1",
        "codigoModalidade": "402101",
    },
    {
        "nome": "Aquisição de veículos",
        "codigoSegmento": "1",
        "codigoModalidade": "401101",
    },
    {
        "nome": "Cartão de crédito parcelado",
        "codigoSegmento": "1",
        "codigoModalidade": "215101",
    },
    {
        "nome": "Cartão de crédito rotativo",
        "codigoSegmento": "1",
        "codigoModalidade": "204101",
    },
    {
        "nome": "Cheque especial",
        "codigoSegmento": "1",
        "codigoModalidade": "216101",
    },
    {
        "nome": "Crédito pessoal consignado INSS",
        "codigoSegmento": "1",
        "codigoModalidade": "218101",
    },
    {
        "nome": "Crédito pessoal consignado privado",
        "codigoSegmento": "1",
        "codigoModalidade": "219101",
    },
    {
        "nome": "Crédito pessoal consignado público",
        "codigoSegmento": "1",
        "codigoModalidade": "220101",
    },
    {
        "nome": "Crédito pessoal não consignado",
        "codigoSegmento": "1",
        "codigoModalidade": "221101",
    },
    {
        "nome": "Desconto de cheques",
        "codigoSegmento": "1",
        "codigoModalidade": "302101",
    },
    {
        "nome": "Financiamento imobiliário com taxas de mercado",
        "codigoSegmento": "1",
        "codigoModalidade": "903101",
    },
    {
        "nome": "Financiamento imobiliário com taxas reguladas",
        "codigoSegmento": "1",
        "codigoModalidade": "905101",
    },
    {
        "nome": "Arrendamento mercantil de veículos",
        "codigoSegmento": "1",
        "codigoModalidade": "1205101",
    },
    {
        "nome": "Antecipação de faturas de cartão de crédito",
        "codigoSegmento": "2",
        "codigoModalidade": "303101",
    },
    {
        "nome": "Capital de giro com prazo até 365 dias",
        "codigoSegmento": "2",
        "codigoModalidade": "210101",
    },
    {
        "nome": "Capital de giro com prazo superior a 365 dias",
        "codigoSegmento": "2",
        "codigoModalidade": "211101",
    },
    {
        "nome": "Cheque especial",
        "codigoSegmento": "2",
        "codigoModalidade": "216101",
    },
    {
        "nome": "Conta garantida",
        "codigoSegmento": "2",
        "codigoModalidade": "217101",
    },
    {
        "nome": "Desconto de cheques",
        "codigoSegmento": "2",
        "codigoModalidade": "302101",
    },
    {
        "nome": "Desconto de duplicata",
        "codigoSegmento": "2",
        "codigoModalidade": "301101",
    },
    {
        "nome": "Vendor",
        "codigoSegmento": "2",
        "codigoModalidade": "404101",
    },
    {
        "nome": "Capital de giro com prazo até 365 dias",
        "codigoSegmento": "2",
        "codigoModalidade": "210204",
    },
    {
        "nome": "Capital de giro com prazo superior a 365 dias",
        "codigoSegmento": "2",
        "codigoModalidade": "211204",
    },
    {
        "nome": "Conta garantida",
        "codigoSegmento": "2",
        "codigoModalidade": "217204",
    },
    {
        "nome": "Adiantamento sobre contratos de câmbio",
        "codigoSegmento": "2",
        "codigoModalidade": "502205",
    },
]


In [ ]:
import requests
import json
from datetime import date, datetime


df_list = []

for produto in PRODUTOS:
    nome = produto["nome"]
    segmento = produto["codigoSegmento"]
    modalidade = produto["codigoModalidade"]
    print(f"START: Extraindo dados do produto --- {nome.upper()}...")

    base = f"https://www.bcb.gov.br/api/servico/sitebcb/historicotaxajurosdiario/TodosCampos?filtro="
    filtro = f"(codigoSegmento eq '{segmento}') and (codigoModalidade eq '{modalidade}') and (InicioPeriodo ge '2025-03-24')"

    url = f"{base}{filtro}"
    try:
        body = requests.get(url=url).text

        json_body = json.loads(body)["conteudo"]


        for item in json_body:
            df_dict = {}
            
            df_dict["NM_BANK"] = item["InstituicaoFinanceira"]
            df_dict["NM_SEGM"] = item["Segmento"]
            df_dict["DS_PROD"] = nome
            df_dict["CD_SEGM"] = segmento
            df_dict["CD_MODL"] = modalidade
            df_dict["NR_POSI"] = item["Posicao"]
            df_dict["DS_MODL"] = item["Modalidade"]
            df_dict["VL_TAXA_JURO_AM"] = float(item["TaxaJurosAoMes"].replace(",","."))
            df_dict["VL_TAXA_JURO_AA"] = float(item["TaxaJurosAoAno"].replace(",","."))
            df_dict["dat_ref_carga"] = str(date.today())
            df_dict["dh_exec"] = datetime.now()

            df_list.append(df_dict)

        print(f"OK: Dados do produto {nome.upper()} extraídos!")
    except Exception as e:
        print(f"ERRO: Falha ao extrair dados do produto --- {nome.upper()} - {e}")

df = spark.createDataFrame(df_list)

print(df.count())
df.show(truncate=False)

In [ ]:
df.show(truncate=False)

In [ ]:
df.select("DS_PROD").distinct().show()

In [ ]:
import pyspark.sql.functions as F

df.filter(F.col("NR_POSI") == 1).show(truncate=False)